In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from scipy.spatial.distance import pdist
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import itertools
import os


from preprocessing.filterHistoricalForecast_v2 import produce_filtered_dataset


In [ ]:
def retrieveData(dist):
    """
    Returns date indexed dataframe of historical forecast data filtered within dist km of turbines
    
    Parameters:
        dist (int): Distance in km from turbines to filter data
    
    Returns:
        df (pd.DataFrame): DataFrame with date as index and sensor data as columns
    """
    
    file_path = f'data/filtered_historicalForecasts/{dist}km_historicalForecast2024.csv'
    print(f'Looking for file at {file_path}...', end=' ')
    if os.path.exists(file_path):
        print(f'FOUND!')
        df = pd.read_csv(file_path)
    else:
        print('File does not exist, creating filtered dataset')
        df = produce_filtered_dataset(
            unfiltered_data_path = 'data/unfiltered_historicalForecast2024.csv',
            turbine_data_path = 'data/uswtdb_v7_2_20241120.csv',
            coordinate_column_path = 'data/coordinate_columns.csv',
            output_folder = 'data/filtered_historicalForecasts',
            max_distance_km = dist
        )
        
    df['Date'] = df['Date'].apply(lambda x: x if " " in x else x + " 00:00:00")
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    
    return df


def fill_missing_hours(df):
    """
    Identifies missing hourly timestamps in a DataFrame and adds rows with NaN values.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame with datetime index or column
        datetime_col (str): Name of datetime column (if not using index)
        
    Returns:
        pd.DataFrame: DataFrame with complete hourly sequence
    """
    
    # Create complete hourly range for the year
    start_date = df.index.min().floor('D')  # Start at beginning of first day
    end_date = df.index.max().ceil('D')     # End at midnight of last day
    full_range = pd.date_range(start=start_date, end=end_date, freq='h', inclusive='left')
    
    # Reindex to add missing hours
    df_complete = df.reindex(full_range)
    
    return df_complete


def getMagDf(dist=10):
    """
    Returns a DataFrame with the magnitude of wind speed for each sensor (forward filled for missing hours)
    
    Parameters:
        dist (int): Distance in km from turbines to filter data
        
    Returns:
        df_mag (pd.DataFrame): DataFrame with date as index and sensor magnitude as columns
    """

    data = retrieveData(dist)

    sensor_ids = sorted(set(col.split('_')[1] for col in data.columns if col.startswith('u80_')))

    mag_columns = {}

    for sensor_id in sensor_ids:
        u_col = f'u80_{sensor_id}'
        v_col = f'v80_{sensor_id}'
        if u_col in data.columns and v_col in data.columns:
            mag_columns[sensor_id] = np.sqrt(data[u_col]**2 + data[v_col]**2)

    # Concatenate all columns at once
    df_mag = pd.DataFrame(mag_columns, index=data.index)
    df_mag = df_mag[sorted(df_mag.columns)]

    df_mag = fill_missing_hours(df_mag).ffill()
        
    return df_mag


def makeKernel(df):
    """
    Creates a kernel covariance matrix for the given DataFrame using Gaussian Process Regression.
        
    Parameters:
        df (pd.DataFrame): DataFrame containing sensor data over time with 1 column per sensor 
        
    Returns:
        K (np.ndarray): Covariance matrix of the kernel
    """
    
    vals = df.values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(vals)
    imputer = SimpleImputer(strategy='median')
    X_imputed = imputer.fit_transform(X_scaled)

    # Heuristic for distance
    dists = pdist(X_scaled.T)  # pairwise distances
    median_dist = np.median(dists)

    kernel = RBF(length_scale=median_dist/2)
    gp = GaussianProcessRegressor(kernel=kernel)
    K = gp.kernel(X_imputed, X_imputed)
    return K

def getCoordinateGrid(df):
    """
    Returns ordered lat and lon dicts with coordinates
    
    Parameters:
        df (pd.DataFrame): DataFrame containing sensor data over time with 1 column per sensor 
        
    Returns:
        lat_dict (dict): Dictionary with sensor IDs as keys and latitude as values
        lon_dict (dict): Dictionary with sensor IDs as keys and longitude as values
    """
    
    sensor_ids = df.columns.astype(int)
            
    df_sensor_coords = pd.read_csv('data/coordinate_columns.csv')
    
    lat_dict = {sensor_id: df_sensor_coords[df_sensor_coords['sensor_id'] == sensor_id]['latitude'].iloc[0] for sensor_id in sensor_ids}
    lon_dict = {sensor_id: df_sensor_coords[df_sensor_coords['sensor_id'] == sensor_id]['longitude'].iloc[0] for sensor_id in sensor_ids}

    return lat_dict, lon_dict


def mutual_info_gain(K_sub, sigma2):
    """
    Computes the mutual information gain for a subset of the covariance matrix
    Args:
        K_sub: Subset of the covariance matrix
        sigma2: Variance of observed values
    Returns:
        mutual_info: Mutual information gain
    """
    n = K_sub.shape[0]
    return 0.5 * np.linalg.slogdet(np.eye(n) + (1 / sigma2) * K_sub)[1]


def greedy_select(K_full, sensor_id_to_index, k=10, sigma2=1e-5):
    """
    Greedy Mutual Information Optimization to select points of maximum information

    Args:
        K_full: kernel matrix (N x N)
        sensor_id_to_index: dict mapping sensor_id to kernel index (0-based)
        k: number of sensors to select
        sigma2: observation noise variance

    Returns:
        selected_ids: list of selected sensor IDs
    """
    remaining_ids = list(sensor_id_to_index.keys())
    selected_ids = []

    for _ in range(k):
        best_gain = -np.inf
        best_sensor = None

        for sensor_id in remaining_ids:
            candidate_ids = selected_ids + [sensor_id]
            candidate_indices = [sensor_id_to_index[sid] for sid in candidate_ids]

            K_sub = K_full[np.ix_(candidate_indices, candidate_indices)]
            gain = mutual_info_gain(K_sub, sigma2)

            if gain > best_gain:
                best_gain = gain
                best_sensor = sensor_id

        selected_ids.append(best_sensor)
        remaining_ids.remove(best_sensor)

    return selected_ids

### caller

In [ ]:
df_mag = getMagDf(5)

#square root to stabilize variance
df_mag_sqrt = np.sqrt(df_mag)
K = makeKernel(df_mag_sqrt)

lat_dict, lon_dict = getCoordinateGrid(df_mag_sqrt)
sensor_id_to_index = {sensor_id: idx for idx, sensor_id in enumerate(df_mag.columns.astype(int))}
k_best_indices = greedy_select(K_full=K,sensor_id_to_index=sensor_id_to_index,k=10)


Looking for file at data/filtered_historicalForecasts/5km_historicalForecast2024.csv... FOUND!
Selecting 1 of 10 sensors
Selected sensor ID: 10077
Selecting 2 of 10 sensors
Selected sensor ID: 3149
Selecting 3 of 10 sensors
Selected sensor ID: 4500
Selecting 4 of 10 sensors
Selected sensor ID: 4797
Selecting 5 of 10 sensors
Selected sensor ID: 17339
Selecting 6 of 10 sensors
Selected sensor ID: 3025
Selecting 7 of 10 sensors
Selected sensor ID: 9000
Selecting 8 of 10 sensors
Selected sensor ID: 4497
Selecting 9 of 10 sensors
Selected sensor ID: 3151
Selecting 10 of 10 sensors
Selected sensor ID: 7532
[10077, 3149, 4500, 4797, 17339, 3025, 9000, 4497, 3151, 7532]
